In [1]:
import pandas as pd
import numpy as np

In [2]:
weather_data_prep = pd.read_csv("weather_data_prep.csv")

## Convert DATE column into Date format

In [3]:
weather_data_prep['DATE']= pd.to_datetime(weather_data_prep['DATE']) 

## Drop useless columns

We think SNOW, SNWD (Snow Depth) and PGTM (Peak gust time) are useless because it's null

In [4]:
weather_data_prep = weather_data_prep.drop(['SNWD', 'PGTM', 'SNOW'], axis=1)

## Aggregate by date

In [5]:
weather_data_prep = weather_data_prep.groupby(['DATE']).agg({
    'AWND': 'mean',
    'PRCP':'mean',
    'TAVG': 'mean',
    'WDF5': 'mean',
    'TMAX':'max',
    'WSF2': 'max',
    'WSF5': 'max',
    'WT01': 'max',
    'WT02': 'max',
    'WT03': 'max',
    'WT08': 'max',
    'TMIN': 'min'
})


weather_data_prep = weather_data_prep.rename(columns={
    "AWND": "AWND_mean", 
    "PRCP": "PRCP_mean", 
    "TAVG": "TAVG_mean", 
    "WDF5": "WDF5_mean", 
    "TMAX": "TMAX_max",
    'WSF2': 'WSF2_max',
    'WSF5': 'WSF5_max',
    'WT01': 'WT01_max',
    'WT02': 'WT02_max',
    'WT03': 'WT03_max',
    'WT08': 'WT08_max',
    'TMIN': 'TMIN_min'
}, errors="raise")








## Add columns

We add two informations that might affect the prediction: if there is snow or rain that day.

In [6]:
weather_data_prep['IceRoad'] = np.where(weather_data_prep['TMIN_min'] < 37.4, 1, 0)
weather_data_prep['WetDay'] = np.where(weather_data_prep['PRCP_mean'] > 0, 1, 0)
weather_data_prep = weather_data_prep.reset_index()

## Load airport dataset

In [7]:
Airport_Data = pd.read_csv("Airport_Data.csv")

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## We drop useless columns


In [8]:
Airport_Data = Airport_Data.drop([
    'stand_last_change', 
    'sto',
    'atot',
    'aobt',
    'chocks_on',
    'stand_scheduled',
    'last_distance_to_gate',
    'last_in_sector',
    'status',
    'mode_s',
    'acReg',
    'partition',
    'vdgs_in',
    'stand_active',
    'stand_docking',
    'aibt_received',
    'sqt',
    'plb_on',
    'pca_on',
    'gpu_on',
    'towbar_on',
    'plb_off',
    'pca_off',
    'gpu_off',
    'acars_out',
    'vdgs_out',
    'stand_free',
    'eobt',
    'aldt_received',
    'stand_prepared',
    'stand_auto_start',
    'roll',
    'speed'
], axis=1)

## We add Date column


In [9]:
Airport_Data['DATE'] = pd.to_datetime(Airport_Data['aldt'], errors='coerce').dt.normalize()

## Merging the two

In [10]:
Airport_and_weather = pd.merge(Airport_Data, weather_data_prep, how='left', on=['DATE'])

In [11]:
Airport_and_weather = Airport_and_weather.rename(columns={
    "carrier": "Airline", 
    "flight": "FlightNumber", 
    "DATE": "Date", 
    "acType": "AircraftType", 
    "ship": "ShipmentWeight",
    'runway': 'Runway',
    'stand': 'Stand',
    'aldt': 'ActualLandingTime',
    'eibt': 'EstimatedInBlockTime',
    'cibt': 'CalculatedInBlockTime',
    'aibt': 'ActualInBlockTime'
}, errors="raise")

## We calculate the Taxi time 

In [12]:
Airport_and_weather['TaxiTime'] = pd.to_datetime(Airport_and_weather['ActualInBlockTime'], errors='coerce') - pd.to_datetime(Airport_and_weather['ActualLandingTime'], errors='coerce')


Airport_and_weather['TaxiTime'] = Airport_and_weather['TaxiTime'].dt.total_seconds() / 60






We drop duplicates and the rows containing NAN values for Taxi time, and the rows containing absurd taxi times values


In [13]:
Airport_and_weather = Airport_and_weather.drop_duplicates()
Airport_and_weather = Airport_and_weather[np.isfinite(Airport_and_weather['TaxiTime'])]
Airport_and_weather = Airport_and_weather[(Airport_and_weather['TaxiTime'] >= 0) & (Airport_and_weather['TaxiTime'] < 120)]



We separate into groups the planes according to their shipment weight, which might affect the taxi time.

In [14]:
conditions = [
    (pd.to_numeric(Airport_and_weather['ShipmentWeight'], errors='coerce') < 2000),
    (pd.to_numeric(Airport_and_weather['ShipmentWeight'], errors='coerce') >= 2000) & (pd.to_numeric(Airport_and_weather['ShipmentWeight'], errors='coerce') < 6000),
    (pd.to_numeric(Airport_and_weather['ShipmentWeight'], errors='coerce') >= 6000)]

choices = ['S', 'M', 'L']

Airport_and_weather['ShipmentWeightCat'] = np.select(conditions, choices)

We also add the number of planes which arrive 10mn before the one in the row. If there is a lot of planes that are landing at the same time, it will affect the taxi time.

In [15]:
Airport_and_weather['ActualLandingTime'] = pd.to_datetime(Airport_and_weather['ActualLandingTime'], errors='coerce')
Airport_and_weather = Airport_and_weather.sort_values(by='ActualLandingTime').set_index('ActualLandingTime')
Airport_and_weather['NbPlanesLast10Mn'] = Airport_and_weather['TaxiTime'].rolling("10T").count()
Airport_and_weather = Airport_and_weather.reset_index()

Airport_and_weather['NbPlanesLast10Mn'] = Airport_and_weather['NbPlanesLast10Mn'] - 1

We add a column with the day of the week (from 0 on monday to 6 on sunday) and the month of the year. 

In [16]:
Airport_and_weather['Hour'] = Airport_and_weather['ActualLandingTime'].dt.hour
Airport_and_weather['DayOfTheWeek'] = Airport_and_weather['ActualLandingTime'].dt.dayofweek

We transform Runway into a numeric values for the model as well as the stand.

In [17]:
Airport_and_weather['Runway'] = pd.to_numeric(Airport_and_weather['Runway'].str[7:9], errors='coerce')
Airport_and_weather = Airport_and_weather.rename(columns={
    "Runway": "RunwayNumber"
}, errors="raise")


Airport_and_weather['Stand'] =  pd.to_numeric(Airport_and_weather['Stand'].str[5:], errors='coerce')

Airport_and_weather = Airport_and_weather.rename(columns={
    "Stand": "StandNumber"
}, errors="raise")

We drop Tmax_max and tmin_min because highly correlated with tavg

In [18]:
Airport_and_weather['FlightNumber'] = pd.to_numeric(Airport_and_weather['FlightNumber'], errors='coerce')

In [19]:
Airport_and_weather = Airport_and_weather.drop([
    'FlightNumber', 
    'EstimatedInBlockTime',
    'CalculatedInBlockTime',
    'ActualInBlockTime',
    'TMAX_max',
    'TMIN_min',
    'ShipmentWeight'
], axis=1)

We calculate the log of the taxi time in order to have a normal distribution.

In [20]:
Airport_and_weather['LogTaxiTime'] = np.log(Airport_and_weather['TaxiTime'] + 1) 

## We now merge with airfacts data. 

Since the name of the aircrafts on the two files provided (ACchar.xlsx & Airport_Data.csv), we created a file containing the same name and the aircrafts specifications.

In [21]:
Aircrafts_info = pd.read_csv("aircraft_simplified_data.csv", sep=";",header=0,encoding = 'unicode_escape')
Aircrafts_info = Aircrafts_info.rename(columns={
    "Model_Airport_Data": "AircraftType"
}, errors="raise")

In [22]:
FinalDataset = pd.merge(Airport_and_weather, Aircrafts_info, how='left', on=['AircraftType'])

We filter dates that had obviously a problem and which are not relevant for the predictions (the average of the taxi time these days are more than 20 minutes)

In [23]:
FinalDataset = FinalDataset[FinalDataset['Date'] != "2018-09-26"]
FinalDataset = FinalDataset[FinalDataset['Date'] != "2018-08-01"]

# Final data preparation 

We rename the columns, drop the columns that are not correlated with parking area(wingspan and length).

In [24]:
# Columns renaming: 
FinalDataset.rename(columns={'Parking Area':'ParkingArea',
                             'WT01_max':'WT01max',
                             'WT02_max':'WT02max', 
                             'WT03_max':'WT03max', 
                             'WT08_max':'WT08max', 
                             'WSF2_max':'WSF2max', 
                             'WSF5_max':'WSF5max',
                             'Model_Acchar':'ModelAcchar',  
                             'TAVG_mean':'TAVGmean',
                             'AWND_mean':'AWNDmean', 
                             'TAVG_mean':'TAVGmean', 
                             'DayOfTheWeek':'WeekDay',
                             'WDF5_mean':'WDF5mean', 
                             'PRCP_mean':'PRCPmean'}, inplace=True)

# Dropping correlated variables:  
FinalDataset=FinalDataset.drop(labels=["Wingspan","Length"], axis=1)

# Converting dates to_datetime: 
FinalDataset.ActualLandingTime=pd.to_datetime(FinalDataset.ActualLandingTime.values)
FinalDataset.Date=pd.to_datetime(FinalDataset.Date.values)


We drop nan values except for the weather where a null just means that it is equal to 0 (example for the rain).

In [25]:

# Dropping (real) NA values ~4%: 
FinalDataset=FinalDataset.dropna(subset=["RunwayNumber", "StandNumber", "AircraftType","ParkingArea"])

# Filling other missing values with 0 : 
FinalDataset=FinalDataset.fillna(0)

# Putting TaxiTime and LogTaxiTime as last columns : 
taxitime=FinalDataset.TaxiTime
logtaxitime=FinalDataset.LogTaxiTime 

FinalDataset.drop(["TaxiTime", "LogTaxiTime", "ModelAcchar"], axis=1, inplace=True)

FinalDataset["TaxiTime"]=taxitime
FinalDataset["LogTaxiTime"]=logtaxitime

In [31]:
export_csv = FinalDataset.to_csv('DatasetCleanFinal.csv', index = None, header=True)

In [36]:
FinalDataset[FinalDataset.isna() == True].count()

ActualLandingTime    0
Airline              0
AircraftType         0
RunwayNumber         0
StandNumber          0
Date                 0
AWNDmean             0
PRCPmean             0
TAVGmean             0
WDF5mean             0
WSF2max              0
WSF5max              0
WT01max              0
WT02max              0
WT03max              0
WT08max              0
IceRoad              0
WetDay               0
ShipmentWeightCat    0
NbPlanesLast10Mn     0
Hour                 0
WeekDay              0
ParkingArea          0
TaxiTime             0
LogTaxiTime          0
dtype: int64

In [28]:
from pandas_profiling import ProfileReport

In [30]:
ProfileReport(FinalDataset)

TemplateSyntaxError: Encountered unknown tag 'with'.